In [3]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import random
import os

### Select representative members for each genus in the Genomoviridae family

In [2]:
# === CONFIG ===
CSV_PATH = "Genomovirus_Proteins.csv"         # Metadata file
FASTA_PATH = "Genomovirus_Proteins.faa"       # Full FASTA file
OUTPUT_FASTA = "rep_subset_by_genus.faa"      # Output file

In [7]:
# === STEP 1: Load and filter metadata ===
df = pd.read_csv(CSV_PATH)

# Keep only rows with "rep" in Protein name (case-insensitive)
df_filtered = df[df['Protein'].str.contains('rep', case=False, na=False)]

# Sample up to 10 accessions per Genus
sampled_accessions = (
    df_filtered.groupby('Genus')['Accession']
    .apply(lambda x: x.sample(n=min(10, len(x)), random_state=42))
    .explode()
    .tolist()
)
sampled_set = set(sampled_accessions)

# === STEP 2: Filter FASTA ===
selected_records = []
for record in SeqIO.parse(FASTA_PATH, "fasta"):
    accession = record.id.split('|')[0].strip()
    if accession in sampled_set and 'rep' in record.description.lower():
        selected_records.append(record)

# === STEP 3: Save output ===
SeqIO.write(selected_records, OUTPUT_FASTA, "fasta")

print(f"✅ Saved {len(selected_records)} Rep sequences to '{OUTPUT_FASTA}'")

✅ Saved 77 Rep sequences to 'rep_subset_by_genus.faa'


In [ ]:
# === CONFIG ===
CSV_PATH = "Genomovirus_Proteins.csv"         # Metadata file
FASTA_PATH = "Genomovirus_Proteins.faa"       # Full FASTA file
OUTPUT_FASTA = "rep_subset_by_genus2.faa"      # Output file

# === STEP 1: Load and filter metadata ===
df = pd.read_csv(CSV_PATH)

# Keep only rows with "rep" in Protein name (case-insensitive)
df_filtered = df[df['Protein'].str.contains('rep', case=False, na=False)]

# Sample up to 10 accessions per Genus
sampled_accessions = (
    df_filtered.groupby('Genus')['Accession']
    .apply(lambda x: x.sample(n=min(10, len(x)), random_state=42))
    .explode()
    .tolist()
)
sampled_set = set(sampled_accessions)

# Make lookup tables for country and genus
metadata_lookup = df_filtered.set_index('Accession')[['Country', 'Genus']].to_dict(orient='index')

# === STEP 2: Filter FASTA and rename headers ===
selected_records = []
for record in SeqIO.parse(FASTA_PATH, "fasta"):
    accession = record.id.split('|')[0].strip()
    if accession in sampled_set and 'rep' in record.description.lower():
        if accession in metadata_lookup:
            # Fix NaN-safe handling
            country = metadata_lookup[accession]['Country']
            genus = metadata_lookup[accession]['Genus']
            country = str(country).strip().replace(" ", "_") if pd.notna(country) else "Unknown"
            genus = str(genus).strip().replace(" ", "_") if pd.notna(genus) else "Unknown"
            
            new_id = f"{accession}_{country}_{genus}"
            record.id = new_id
            record.name = new_id
            record.description = ""
            selected_records.append(record)

# === STEP 3: Save output ===
SeqIO.write(selected_records, OUTPUT_FASTA, "fasta")
print(f"✅ Saved {len(selected_records)} renamed Rep sequences to '{OUTPUT_FASTA}'")


✅ Saved 77 renamed Rep sequences to 'rep_subset_by_genus2.faa'


## Extract Rep from Gb Files

In [7]:
# === CONFIG ===
input_gb = "Genomovirus.gb"                      # Input GenBank file
output_fna = "rep_genes_nucleotide.fna"              # Output FASTA

rep_cds_records = []

for record in SeqIO.parse(input_gb, "genbank"):
    accession = record.id
    genus = "Unknown"
    country = "Unknown"

    # === STEP 1: Try to get genus and country from source feature ===
    for feature in record.features:
        if feature.type == "source":
            genus = feature.qualifiers.get("genus", ["Unknown"])[0]
            country = feature.qualifiers.get("geo_loc_name", ["Unknown"])[0]
            break

    # === STEP 2: Fallback to taxonomy line for genus if not found ===
    if genus == "Unknown":
        taxonomy = record.annotations.get("taxonomy", [])
        if taxonomy:
            genus = taxonomy[-1]

    # === STEP 3: Extract Rep CDS features ===
    for feature in record.features:
        if feature.type == "CDS" and "product" in feature.qualifiers:
            product = feature.qualifiers["product"][0].lower()
            if "rep" in product:
                rep_seq = feature.extract(record.seq)
                header = f"{accession}_{genus}_{country}".replace(" ", "_")
                rep_cds_records.append(SeqRecord(
                    rep_seq, id=header, description="Rep CDS"))

# === OUTPUT ===
SeqIO.write(rep_cds_records, output_fna, "fasta")
print(f"✅ Extracted {len(rep_cds_records)} Rep CDS sequences to '{output_fna}'")

✅ Extracted 2311 Rep CDS sequences to 'rep_genes_nucleotide.fna'


In [10]:
# === CONFIG ===
input_gb = "Genomovirus.gb"                      # Input GenBank file
output_fna = "rep_genes_nucleotide.fna"              # Output FASTA file

rep_gene_records = []

for record in SeqIO.parse(input_gb, "genbank"):
    accession = record.id
    genus = "Unknown"
    country = "Unknown"

    # === STEP 1: Extract genus and country from 'source' feature ===
    for feature in record.features:
        if feature.type == "source":
            genus = feature.qualifiers.get("genus", ["Unknown"])[0]
            country = feature.qualifiers.get("geo_loc_name", ["Unknown"])[0]
            break

    # === STEP 2: Fallback to taxonomy line for genus ===
    if genus == "Unknown":
        taxonomy = record.annotations.get("taxonomy", [])
        if taxonomy:
            genus = taxonomy[-1]

    # === STEP 3: Extract gene features with "rep" in gene name ===
    for feature in record.features:
        if feature.type == "gene" and "gene" in feature.qualifiers:
            gene_name = feature.qualifiers["gene"][0].lower()
            if "rep" in gene_name:
                rep_seq = feature.extract(record.seq)
                header = f"{accession}_{genus}_{country}".replace(" ", "_")
                rep_gene_records.append(SeqRecord(
                    rep_seq, id=header, description=""))

# === OUTPUT ===
SeqIO.write(rep_gene_records, output_fna, "fasta")
print(f"✅ Extracted {len(rep_gene_records)} Rep gene sequences to '{output_fna}'")

✅ Extracted 234 Rep gene sequences to 'rep_genes_nucleotide.fna'


## subsample rep nuc seqeunces from previous step

In [4]:
input_fasta = "rep_genes_nucleotide.fna"
output_fasta = "subsampled.fasta"

def extract_genus(header):
    parts = header.strip(">").split("_")
    if parts[0].startswith("NC"):
        return parts[2] if len(parts) > 2 else None
    else:
        return parts[1] if len(parts) > 1 else None

# Read sequences and group by genus
genus_to_seqs = defaultdict(list)

for record in SeqIO.parse(input_fasta, "fasta"):
    genus = extract_genus(record.description)
    if genus:
        genus_to_seqs[genus].append(record)

# Subsample and write
with open(output_fasta, "w") as out_f:
    for genus, seqs in genus_to_seqs.items():
        for record in seqs[:5]:  # Keep only first 5
            SeqIO.write(record, out_f, "fasta")